In [4]:
!pip3 install pyspark

  Using cached pyspark-3.3.0.tar.gz (281.3 MB)
  Preparing metadata (setup.py) ... done
  Using cached py4j-0.10.9.5-py2.py3-none-any.whl (199 kB)
Using legacy 'setup.py install' for pyspark, since package 'wheel' is not installed.
  Running setup.py install for pyspark ... done
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [5]:
!alias python=python3

In [6]:
!python --version

Python 3.9.12


# PySpark : Excercies RDD

In [8]:
from pyspark.sql import SparkSession

In [9]:
spark = SparkSession.builder.appName("MyApp").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/22 00:29:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JError: An error occurred while calling None.org.apache.spark.sql.SparkSession. Trace:
py4j.Py4JException: Constructor org.apache.spark.sql.SparkSession([class org.apache.spark.SparkContext, class java.util.HashMap]) does not exist
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:179)
	at py4j.reflection.ReflectionEngine.getConstructor(ReflectionEngine.java:196)
	at py4j.Gateway.invoke(Gateway.java:237)
	at py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
	at py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.lang.Thread.run(Thread.java:748)



In [10]:
rdd_context = spark.sparkContext.textFile("./datas/datas.txt")

In [11]:
rdd_context.collect()

['mon joli fichier texte du futur']

## Map

In [12]:
rdd_file = rdd_context.map(lambda x: x.upper())

In [13]:
print(rdd_file.collect())

['MON JOLI FICHIER TEXTE DU FUTUR']


## FlatMap

In [14]:
flatmap = ["mot1 mot2 ", "mot3 mot4 "," mot5 mot6 "]

In [15]:
rdd_flatmap = spark.sparkContext.parallelize(flatmap)

In [16]:
for flatten in rdd_flatmap.collect():
  print(flatten)

mot1 mot2 
mot3 mot4 
 mot5 mot6 


In [17]:
rdd_flatmap2 = rdd_flatmap.flatMap(lambda x: x.split(" "))
for flatten2 in rdd_flatmap2.collect():
  print(flatten2)

mot1
mot2

mot3
mot4


mot5
mot6



In [18]:
def mapplify_mapflatify(datas):
  rdd_context = spark.sparkContext.parallelize(datas)
  rdd_mapped = rdd_context.map(lambda x: x)
  rdd_flatmapped = rdd_mapped.flatMap(lambda x: x)
  for element in rdd_flatmapped.collect():
    print(element)

In [19]:
mapplify_mapflatify(flatmap)

m
o
t
1
 
m
o
t
2
 
m
o
t
3
 
m
o
t
4
 
 
m
o
t
5
 
m
o
t
6
 


## Filter

In [20]:
filter_datas = ["Pierre", "Alpha", "Mehdi", "Jean-Pierre", "Thierry"]

In [21]:
def find_composed_name(datas):
  filter_rdd_context = spark.sparkContext.parallelize(datas)
  composed_name = filter_rdd_context.filter(lambda x: "-" in x).collect()
  return composed_name

In [22]:
find_composed_name(filter_datas)

['Jean-Pierre']

## Distinct

In [23]:
listed_names = ["Pierre", "Alpha", "Mehdi", "Jean-Pierre", "Thierry", "Pierre"]

In [24]:
def show_distinct(datas):
  disctinct_rdd_context = spark.sparkContext.parallelize(datas)
  disctinted_datas = disctinct_rdd_context.distinct()
  
  return disctinted_datas.collect()

In [25]:
show_distinct(listed_names)

['Pierre', 'Alpha', 'Mehdi', 'Jean-Pierre', 'Thierry']

## GroupBy

In [26]:
def group_names(datas):
  groupnames_rdd_context = spark.sparkContext.parallelize(datas)
  # Regrouper sur chaque premier élément de chaque caractère de la liste 
  groupnames_datas = groupnames_rdd_context.groupBy(lambda x: x == "P") \
    .mapValues(list) \
    .collect()

  print(groupnames_datas)

In [27]:
group_names(listed_names)

[(False, ['Pierre', 'Alpha', 'Mehdi', 'Jean-Pierre', 'Thierry', 'Pierre'])]


## Sample

In [28]:
def sampling(datas):
  sampling_rdd_context = spark.sparkContext.parallelize(datas)
  sampling_datas = sampling_rdd_context.sample(0.5, 9).collect()
  
  return sampling_datas

In [29]:
sampling(listed_names)

['Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Alpha',
 'Mehdi',
 'Mehdi',
 'Mehdi',
 'Mehdi',
 'Mehdi',
 'Mehdi',
 'Mehdi',
 'Mehdi',
 'Jean-Pierre',
 'Jean-Pierre',
 'Jean-Pierre',
 'Jean-Pierre',
 'Jean-Pierre',
 'Jean-Pierre',
 'Jean-Pierre',
 'Jean-Pierre',
 'Thierry',
 'Thierry',
 'Thierry',
 'Thierry',
 'Thierry',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre',
 'Pierre']

# Programmation RDDs

## Union

In [30]:
liste1 = [1, 2, 3]
liste2 = [3, 4, 5]

In [31]:
rdd1 = spark.sparkContext.parallelize(liste1)
rdd2 = spark.sparkContext.parallelize(liste2)

In [32]:
unionDF = rdd1.union(rdd2)

In [33]:
unionDF.collect()

[1, 2, 3, 3, 4, 5]

## Substract

In [34]:
rdd1.subtract(rdd2).collect()

[1, 2]

In [35]:
rdd2.subtract(rdd1).collect()

[4, 5]

## Cartesien

In [36]:
rdd1.cartesian(rdd2).collect()

[(1, 3), (1, 4), (1, 5), (2, 3), (3, 3), (2, 4), (2, 5), (3, 4), (3, 5)]

In [37]:
rdd2.cartesian(rdd1).collect()

[(3, 1), (3, 2), (3, 3), (4, 1), (5, 1), (4, 2), (4, 3), (5, 2), (5, 3)]

## Reduce

In [38]:
listed_rdd = range(1, 6)

In [39]:
listed_ctx_rdd = spark.sparkContext.parallelize(listed_rdd)

In [40]:
listed_ctx_rdd.reduce(lambda x, y: x + y)

15

# Programmation RDDs Advanced 

## RDD Pair : `Map()` and `KeyBy()`

In [41]:
key_pair_ds = ["cle1 valeur1", "cle2 valeur2", "cle3 valeur3"]

In [42]:
def map_keyyed(datas):
  transform_datas = spark.sparkContext.parallelize(datas)
  rdd_flatmapped = transform_datas.map(
      lambda x:
      (x, 1)
  )
  rdd_keyby = rdd_flatmapped.keyBy(lambda x: (x, 1)).collect()
  print(rdd_keyby)

  #return rdd_keyby

In [43]:
map_keyyed(key_pair_ds)

[((('cle1 valeur1', 1), 1), ('cle1 valeur1', 1)), ((('cle2 valeur2', 1), 1), ('cle2 valeur2', 1)), ((('cle3 valeur3', 1), 1), ('cle3 valeur3', 1))]


## `GroupByKey`

In [44]:
to_regroup = ["0, 11", "1, 11", "0, 4", "2, 8", "1, 1", "9, 8"]

In [64]:
def regroup_datas(datas):
  ctx_transform_datas = spark.sparkContext.parallelize(datas)
  rdd_grouped = ctx_transform_datas.groupByKey().mapValues(list).collect()

  return rdd_grouped

In [65]:
# regroup_datas(to_regroup)

Py4JJavaError: ignored

## reduceByKey